In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


Q2 : What is the average height of the players?

In [ ]:

# Reading the file csv
df_1 = pd.read_csv(f"tennis_data/away_team.csv")
df_2 = pd.read_csv(f"tennis_data/home_team.csv")

# Concatenate two csv files
merge_data = pd.concat([df_1,df_2], ignore_index=True)

# Filtering the available data based on needs and removing duplicate data and nan data in the height column
final_data = merge_data[["match_id", "name", "height"]].drop_duplicates(subset='name').dropna(subset=['height'])

# Calculate the mean height of the players
mean_height = final_data['height'].mean()
'The mean height of the players is '+ str(mean_height) + ' ' + 'm'

Q8 :Is there a difference in the number of double faults based on gender?

In [ ]:
# Reading the file csv
df_away_team = pd.read_csv(f"tennis_data/away_team.csv")
df_home_team = pd.read_csv(f"tennis_data/home_team.csv")
df_period = pd.read_csv('tennis_data/period.csv')

# Concatenate two csv files and filtering on them
players = pd.concat(
    [df_away_team[['match_id','gender']],
     df_home_team[['match_id','gender']]])

# Filtering and finding double faults
df_double_faults =df_period[df_period['statistic_name'] == 'double_faults']

# Merging player and double_faults files and filtering on all sets
df_final = players.merge(df_double_faults[['match_id', 'home_value','away_value','period']], on='match_id')
df_final = df_final[df_final['period'] == 'ALL']

# Removing duplicate games and add 'home_value' and 'away_value' columns together and store in a new column 'sum_double_faults'
df_final.drop_duplicates(subset='match_id',inplace=True)
df_final['sum_double_fault'] = df_final['home_value'] + df_final['away_value']

# Group the data by 'gender' and generate descriptive statistics for 'sum_double_fault'
result = df_final.groupby('gender')['sum_double_fault'].describe()
result

Q12 : What is the average number of games per set in men's matches compared to women's matches?

In [ ]:
# Load and filter match statistics data from period.csv 
df_period = pd.read_csv('tennis_data/period.csv')
df_game= df_period[['match_id', 'period', 'statistic_name', 'home_value', 'away_value']]
df_game = df_game[df_period['statistic_name'] == 'total_won']
df_game = df_game[df_period['period'] == 'ALL']

# Add total_game column by summing home and away values
df_game['total_game'] = df_game['home_value'] + df_game['away_value']

# Merge home and away team data to create unified gender column per match
df_home = pd.read_csv('tennis_data/home_team.csv')[['match_id', 'gender']]
df_away = pd.read_csv('tennis_data/away_team.csv')[['match_id', 'gender']]
df_gender = pd.merge(df_home, df_away , on='match_id', suffixes=('_home', '_away'))
df_gender = df_gender[df_gender['gender_home'] == df_gender['gender_away']]
df_gender['gender'] = df_gender['gender_home']

# Merge game and gender data to create final dataset and remove duplicate matches
df_final = df_game.merge(df_gender[['match_id', 'gender']],on='match_id',how='inner')
df_final.drop_duplicates(subset='match_id',inplace=True)

# Extract and count number of sets (1st, 2nd, 3rd) per match from period data
df_set_1ST = df_period[df_period['period'].str.contains("1ST",case=False,na=False)] \
        .groupby('match_id')['period'].nunique().reset_index(name='num_set')
df_set_2ND = df_period[df_period['period'].str.contains("2ND",case=False,na=False)] \
        .groupby('match_id')['period'].nunique().reset_index(name='num_set')
df_set_3RD = df_period[df_period['period'].str.contains("3RD",case=False,na=False)] \
        .groupby('match_id')['period'].nunique().reset_index(name='num_set')

# Combine and sum set counts from 1st, 2nd, and 3rd sets per match
counts_set = pd.concat([df_set_1ST,df_set_2ND,df_set_3RD])
counts_set = counts_set.groupby('match_id').sum('num_set')

# Merge set counts with match data and calculate average games per set by gender
df_final = df_final.merge(counts_set,on='match_id')
df_final['avg_game_set'] = df_final['total_game'] / df_final['num_set']
result = df_final.groupby('gender')['avg_game_set'].describe()
result





Q14 : What is the most common type of surface used in tournaments?

In [ ]:
# Read match_id and ground_type columns from tournament.csv to get the court surface information
df_tournament = pd.read_csv('tennis_data/tournament.csv')[['match_id','ground_type']]

# Remove rows with missing ground_type values to ensure valid surface data
df_tournament.dropna(subset='ground_type', inplace=True)

# Count how many matches were played on each ground type
df_ground_type = df_tournament['ground_type'].value_counts().to_frame().reset_index()

# Calculate the percentage of each ground type relative to total matches
df_ground_type['Percentage'] = (df_ground_type['count'] / df_ground_type['count'].sum() * 100).round(2)

df_ground_type




Q16: Which player has the highest winning percentage against top 10 ranked opponents?

In [ ]:
# Read specific columns from each CSV file:
# - 'home_team.csv': contains home players' info (match ID, name, ID, rank, gender)
# - 'away_team.csv': contains away players' info (match ID, name, ID, rank, gender)
# - 'event.csv': contains match-level data with winner information
df_home = pd.read_csv('tennis_data/home_team.csv')[['match_id','full_name','player_id','current_rank','gender']]
df_away = pd.read_csv('tennis_data/away_team.csv')[['match_id','full_name','player_id','current_rank','gender']]
df_event = pd.read_csv('tennis_data/event.csv')[['match_id','winner_code']]



# Remove rows from home and away DataFrames where 'current_rank' or 'gender' is missing (NaN)
df_home.dropna(subset=['current_rank', 'gender'],inplace=True)
df_away.dropna(subset=['current_rank', 'gender'],inplace=True)




# Merge home and away player data on 'match_id' to create a combined dataset for each match
# Add suffixes '_home' and '_away' to distinguish columns from each side
players = df_home.merge(df_away,on='match_id',suffixes=('_home','_away'))



# Filter matches where both players are male and make a copy of that subset
men_players = players[(players['gender_home'] == "M") & (players['gender_away'] == "M")].copy()
# Filter matches where both players are female and make a copy of that subset
woman_players = players[(players['gender_home'] == "F") & (players['gender_away'] == "F")].copy()



# Merge male players' match data with event information to include the winner code
matches_men = men_players.merge(df_event, on='match_id')

# Remove duplicate match records to ensure one entry per match
matches_men.drop_duplicates(subset='match_id', inplace=True)

# Remove matches with missing winner information
matches_men.dropna(subset=['winner_code'], inplace=True)

# Create boolean columns indicating if home/away player faced a top-10 opponent
matches_men['home_vs_top10'] = matches_men['current_rank_away'] <= 10
matches_men['away_vs_top10'] = matches_men['current_rank_home'] <= 10

# Mark matches where the home/away player won against a top-10 opponent
matches_men['home_is_winner'] = (matches_men['winner_code'] == 1) & (matches_men['current_rank_away'] <= 10)
matches_men['away_is_winner'] = (matches_men['winner_code'] == 2) & (matches_men['current_rank_home'] <= 10)

# Aggregate stats for home male players against top-10 opponents
home_men_vs_top10 = matches_men[matches_men['home_vs_top10']].groupby(
    ['player_id_home', 'full_name_home']
).agg(
    total_match_vs_top10=('match_id', 'nunique'),  # Count unique matches
    wins_vs_top10=('home_is_winner', 'sum')       # Sum wins against top-10
).reset_index()

# Aggregate stats for away male players against top-10 opponents
away_men_vs_top10 = matches_men[matches_men['away_vs_top10']].groupby(
    ['player_id_away', 'full_name_away']
).agg(
    total_match_vs_top10=('match_id', 'nunique'),  # Count unique matches
    wins_vs_top10=('away_is_winner', 'sum')       # Sum wins against top-10
).reset_index()

# Rename columns for consistency across home/away datasets
home_men_vs_top10.rename(columns={
    'player_id_home': 'player_id',
    'full_name_home': 'full_name'
}, inplace=True)

away_men_vs_top10.rename(columns={
    'player_id_away': 'player_id',
    'full_name_away': 'full_name'
}, inplace=True)

# Combine home and away stats into a single male players dataset
men_players_vs_top10 = pd.concat([home_men_vs_top10, away_men_vs_top10]).groupby(
    ['player_id', 'full_name']
).sum().reset_index()

# Calculate win percentage against top-10 opponents and round
men_players_vs_top10['Percentage'] = (
    men_players_vs_top10['wins_vs_top10'] / men_players_vs_top10['total_match_vs_top10'] * 100
).round()

# Sort male players by win percentage against top-10 opponents in descending order
men_players_vs_top10.sort_values(by='Percentage', ascending=False, inplace=True)





# Merge female players' match data with event information to include the winner code
matches_woman = players.merge(df_event, on='match_id')

# Remove duplicate match records to ensure one entry per match
matches_woman.drop_duplicates(subset='match_id', inplace=True)

# Remove matches with missing winner information
matches_woman.dropna(subset=['winner_code'], inplace=True)

# Create boolean columns indicating if home/away player faced a top-10 opponent
matches_woman['home_vs_top10'] = matches_woman['current_rank_away'] <= 10
matches_woman['away_vs_top10'] = matches_woman['current_rank_home'] <= 10

# Mark matches where the home/away player won against a top-10 opponent
matches_woman['home_is_win'] = (matches_woman['winner_code'] == 1) & (matches_woman['current_rank_away'] <= 10)
matches_woman['away_is_win'] = (matches_woman['winner_code'] == 2) & (matches_woman['current_rank_home'] <= 10)

# Aggregate stats for home female players against top-10 opponents
home_woman_vs_top10 = matches_woman[matches_woman['home_vs_top10']].groupby(
    ['player_id_home', 'full_name_home']
).agg(
    total_match_vs_top10=('match_id', 'nunique'),  # Count unique matches
    wins_vs_top10=('home_is_win', 'sum')          # Sum wins against top-10
).reset_index()

# Aggregate stats for away female players against top-10 opponents
away_woman_vs_top10 = matches_woman[matches_woman['away_vs_top10']].groupby(
    ['player_id_away', 'full_name_away']
).agg(
    total_match_vs_top10=('match_id', 'nunique'),  # Count unique matches
    wins_vs_top10=('away_is_win', 'sum')          # Sum wins against top-10
).reset_index()

# Rename columns for consistency across home/away datasets
home_woman_vs_top10.rename(columns={
    'player_id_home': 'player_id',
    'full_name_home': 'full_name'
}, inplace=True)

away_woman_vs_top10.rename(columns={
    'player_id_away': 'player_id',
    'full_name_away': 'full_name'
}, inplace=True)

# Combine home and away stats into a single female players dataset
woman_players_vs_top10 = pd.concat([home_woman_vs_top10, away_woman_vs_top10]).groupby(
    ['player_id', 'full_name']
).sum().reset_index()

# Calculate win percentage against top-10 opponents and round
woman_players_vs_top10['Percentage'] = (
    woman_players_vs_top10['wins_vs_top10'] / woman_players_vs_top10['total_match_vs_top10'] * 100
).round()

# Sort female players by win percentage against top-10 opponents in descending order
woman_players_vs_top10.sort_values(by='Percentage', ascending=False, inplace=True)



Q6 : Which country has produced the most successful tennis players?

In [ ]:
# Read away\home team data with required columns
df_away_team = pd.read_csv(f"tennis_data/away_team.csv")[['match_id','full_name','country']]
df_home_team = pd.read_csv(f"tennis_data/home_team.csv")[['match_id','full_name','country']]

# Read event data including winner code
df_event = pd.read_csv(f"tennis_data/event.csv")

# Add a column to distinguish team type
df_away_team['team_type'] = 'away'
df_home_team['team_type'] = 'home'

# Merge home and away team data on match_id; use suffixes to distinguish home/away columns and remove duplicate matches
players = df_home_team.merge(df_away_team, on='match_id', suffixes=['_home','_away'], how='outer')
players.drop_duplicates(subset='match_id', inplace=True)

# Merge players with event results to include winner information and drop rows with missing winner or country information
merge_df = players.merge(df_event[['match_id', 'winner_code']], on='match_id')
merge_df.dropna(subset=['winner_code','country_home','country_away'], inplace=True)

# Create boolean columns to indicate if home/away player won
merge_df['is_winner_home'] = (merge_df['winner_code'] == 1) & (merge_df['team_type_home'] == 'home')
merge_df['is_winner_away'] = (merge_df['winner_code'] == 2) & (merge_df['team_type_away'] == 'away')

# Remove duplicates after winner calculations
merge_df.drop_duplicates(subset='match_id', inplace=True)

# Aggregate wins for home players
final_df_home = merge_df.groupby(['match_id', 'full_name_home', 'country_home']).agg(
    wins=('is_winner_home', 'sum')
).reset_index()

# Aggregate wins for away players
final_df_away = merge_df.groupby(['match_id', 'full_name_away', 'country_away']).agg(
    wins=('is_winner_away', 'sum')
).reset_index()

# Standardize column names to combine home and away stats
final_df_home.rename(columns={'full_name_home':'full_name','country_home':'country'}, inplace=True)
final_df_away.rename(columns={'full_name_away':'full_name','country_away':'country'}, inplace=True)

# Combine home and away stats, calculate total matches and wins per country
final_df = pd.concat([final_df_home, final_df_away]).groupby('country').agg(
    total_match=('match_id','nunique'),
    total_wins=('wins','sum')
).reset_index()

# Sort countries by total wins
final_df.sort_values(by='total_wins', ascending=False, inplace=True)

# Calculate win percentage per country
final_df['Percentage'] = (final_df['total_wins'] / final_df['total_match'] * 100).round(2)

# Keep top 10 countries
final_df = final_df.head(10)

# Sort top countries by win percentage
final_df.sort_values(by='Percentage', ascending=False, inplace=True)

# Reset index to start from 1
final_df = final_df.reset_index(drop=True)
final_df.index = final_df.index + 1
